# Dataset for Andy C. of a series of 5s images on either side of focus with dz=0.8mm

In [ ]:
import sys
import asyncio
import time
import os

import numpy as np
import logging 
import yaml
import matplotlib.pyplot as plt
import astropy

from lsst.ts import salobj
from lsst.ts.externalscripts.auxtel.latiss_cwfs_align import LatissCWFSAlign
from lsst.ts.observatory.control.utils import RotType

from lsst.ts.idl.enums.Script import ScriptState

from lsst.ts.observing.utilities.decorated_logger import DecoratedLogger

In [ ]:
logger = DecoratedLogger.get_decorated_logger()
logger.level = logging.DEBUG

## Instantiate the Script

First derive and index for the script that is specific to your user

In [ ]:
logger.info(f'Your UID is {os.getuid()}')
index=os.getuid()*10+np.random.randint(0,9)
logger.info(f'The generated index is {index}')

Instantiate the script then start all remotes

In [ ]:
script = LatissCWFSAlign(index=index, remotes=True)  # this essentially calls the init method
await script.start_task

Set the script to have a DEBUG log level

In [ ]:
script.log.level = logging.DEBUG

### Slew to a target to focus the telescope
This is not part of the actual script but seems to come in handy, so the code is here in the notebook for convenience

Print the available instrument filter and gratings. <br>
This is useful both for slewing and for the configuration steps below

In [ ]:
inst_setup = await script.latiss.get_available_instrument_setup()
logger.info(f'filters are: {inst_setup[0]},\ngratings are: {inst_setup[1]}')

Uncomment the following line to slew

#### Find/Specify a target based on the Az/El and slew to it

In [ ]:
# target = await atcs.find_target(160,60,mag_limit=6.0)

In [ ]:
target = "" 

In [ ]:
await script.atcs.slew_object(target, rot_type=RotType.PhysicalSky)

Check that the target is in the field

In [ ]:
await script.latiss.take_engtest(2)

Offset the telescope (if required). <br>
Offsets are in detector X/Y coordinates and in arcseconds

In [ ]:
await script.atcs.offset_xy(x=0, y=-50)

## Configure the script

In [ ]:
configuration = yaml.safe_dump({"filter": 'FELH0600', 
                                "grating": 'empty_1',
                                "exposure_time": 20,})

Set script state to UNCONFIGURED. <br>
The next line is not required the first time the script is run, however, in each additional instance the cell is run, an error will be thrown if it is not included. <br>
Therefore, it is included here despite being a non-operation in the first instance.

In [ ]:
script.set_state(ScriptState.UNCONFIGURED)

#### Put the ScriptState to CONFIGURED

In [ ]:
config_data = script.cmd_configure.DataType()
config_data.config = configuration
await script.do_configure(config_data)

Set these script parameters to None to verify a re-reduction does not happen of the images

In [ ]:
script.intra_visit_id = None
script.extra_visit_id = None
script.short_timeout = 10

### Set groupID and launch the script
This sets the same group ID for all exposures taken in the script

In [ ]:
group_id_data = script.cmd_setGroupId.DataType(
                groupId=astropy.time.Time.now().isot
            )
await script.do_setGroupId(group_id_data)
await script.arun()

## Telescope is now collimated and in focus

#### Grab focus offsets incase we need to come back into focus

In [ ]:
original_focus_offset = await script.atcs.rem.ataos.evt_focusOffsetSummary.aget()
print(original_focus_offset)

## Adjust hexapod for intra-focal images
This hexapod offset pushes it towards M1, therefore pushing the focus behind the detector.
This means we're capturing the extra-focal image.

In [ ]:
await script.rem.ataos.cmd_offset.set_start(z=0.8)

In [ ]:
intra_focal_offsets = await script.atcs.rem.ataos.evt_focusOffsetSummary.aget()
print(intra_focal_offsets)

### Take a stack of 20 x 5 sec intra-focal images in FELH0600
Andy asked for 18 but sometimes the mount can do funny things

In [ ]:
intra_focal_images = await script.latiss.take_engtest(
    exptime=5, n=20, filter='FELH0600', grating='empty_1')

print('Intra focal images= ' + str(intra_focal_images))

## Adjust hexapod for in-focus images

In [ ]:
await script.rem.ataos.cmd_offset.set_start(z=-0.8)

In [ ]:
in_focus_offsets = await script.atcs.rem.ataos.evt_focusOffsetSummary.aget()
print(in_focus_offsets)

### Take a stack of 20 x 5 sec in-focus images in FELH0600
Andy asked for 18 but sometimes the mount can do funny things

In [ ]:
in_focus_images = await script.latiss.take_engtest(
    exptime=5, n=20, filter='FELH0600', grating='empty_1')

print('In focus images= ' + str(in_focus_images))

## Adjust hexapod for extra-focal images
This hexapod offset pulls it away from M1, therefore placing the focus in-front of the detector.
This means it's the extra-focal image.

In [ ]:
await script.rem.ataos.cmd_offset.set_start(z=-0.8)

In [ ]:
extra_focal_offsets = await script.atcs.rem.ataos.evt_focusOffsetSummary.aget()
print(extra_focal_offsets)

### Take a stack of 20 x 5 sec images in FELH0600
Andy asked for 18 but sometimes the mount can do funny things

In [ ]:
extra_focal_images = await script.latiss.take_engtest(
    exptime=5, n=20, filter='FELH0600', grating='empty_1')

print ('FELH0600 stack Sequence_image = ' + str(science_image))

## Put hexapod back to the focus position

In [ ]:
await script.rem.ataos.cmd_offset.set_start(z=0.8)

In [ ]:
in_focus_offsets2 = await script.atcs.rem.ataos.evt_focusOffsetSummary.aget()
print(in_focus_offsets2)

### Check that the focus offsets are the same as when we started

In [ ]:
assert (in_focus_offsets2.userApplied - original_focus_offset.userApplied) < 2e-3

Take a quick snap to verify things are ok

In [ ]:
await script.latiss.take_engtest(exptime=5, n=1, filter='FELH0600', grating='empty_1')